In [1]:
import os
import torch
import mlflow
from gpt import GPTLanguageModel, GPT2Extended
from gpt_params import tokenizer

/Users/joaquinpolonuer/Documents/venvs/ml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials.json"
mlflow.set_tracking_uri(uri="http://34.176.94.221:5000")


In [3]:
device = (
    "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
)

experiment = "Transformer"
run_id = "17825e172dbe48b8934c57b59e85d83c"

gpt_model = GPTLanguageModel.load_from_mlflow(experiment, run_id, device)
gpt_2_model = GPT2Extended.from_pretrained("gpt2").to(device)

/Users/joaquinpolonuer/Documents/exactas/nlp/NLP-monosemanticity/gpt.py:151: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(local_model_path, map_location=

In [5]:
gpt_model.eval()
idx = (
    torch.tensor(tokenizer.encode("The quick brown fox jumps over the lazy dog"), dtype=torch.long)
    .unsqueeze(0)
    .to(device)
)
out = gpt_model.generate(idx, 100)

print(tokenizer.decode(out.squeeze().tolist()))

The quick brown fox jumps over the lazy dogtown system . 
 Despite the foundation naming it proved somewhat ongoing . eighteenth century , his right @-@ time and seven fires during the farm , was suspected this lighting population as would be better known for spirits or fighting funding . In September 1830 of the Good Republic , Charleston , formed an revoked superstructure . Several portion of the roofs were selected on private waters , and the valley were moved west of Bar 275 , or photrastatingrumouthis mostly , he was placed in a large number of


In [6]:
gpt_model.eval()
idx = (
    torch.tensor(tokenizer.encode("The quick brown fox jumps over the lazy dog"), dtype=torch.long)
    .unsqueeze(0)
    .to(device)
)
out = gpt_model.greedy_generate(idx, 100)

print(tokenizer.decode(out.squeeze().tolist()))

The quick brown fox jumps over the lazy dog . The first @-@ time , and the other hand , and the other hand @-@ shaped , and the other hand @-@ shaped , and the other two @-@ year @-@ old @-@ class aircraft . 
 = = = = = 
 The first @-@ year @-@ old @-@ class ships were the first @-@ year @-@ old @-@ class ships . The ship was the first @-@ time in


In [16]:
sample_output = gpt_2_model.generate(
    idx,
    max_new_tokens=40,
    do_sample=False,
)
print(tokenizer.decode(sample_output.squeeze().tolist()))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The quick brown fox jumps over the lazy dog and runs off.

"I'm sorry, I'm sorry, I'm sorry," the fox says.

"I'm sorry, I'm sorry, I'm sorry," the fox


The quick brown fox jumps over the lazy dog who didn't recognize him and spits his meat into the helpless hound's gut. She crawls toward the door, straightened the back of her pants, and opens it. The fox bur
